<h2 id="50-データの入手整形">50. データの入手・整形</h2>
<p><a href="https://archive.ics.uci.edu/ml/datasets/News+Aggregator">News Aggregator Data Set</a>をダウンロードし、以下の要領で学習データ（<code class="language-plaintext highlighter-rouge">train.txt</code>），検証データ（<code class="language-plaintext highlighter-rouge">valid.txt</code>），評価データ（<code class="language-plaintext highlighter-rouge">test.txt</code>）を作成せよ．</p>
<ol>
<li>ダウンロードしたzipファイルを解凍し，<code class="language-plaintext highlighter-rouge">readme.txt</code>の説明を読む．</li>
<li>情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．</li>
<li>抽出された事例をランダムに並び替える．</li>
<li>抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれ<code class="language-plaintext highlighter-rouge">train.txt</code>，<code class="language-plaintext highlighter-rouge">valid.txt</code>，<code class="language-plaintext highlighter-rouge">test.txt</code>というファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．</li>
</ol>
<p>学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．</p>


<h2 id="51-特徴量抽出">51. 特徴量抽出</h2>
<p>学習データ，検証データ，評価データから特徴量を抽出し，それぞれ<code class="language-plaintext highlighter-rouge">train.feature.txt</code>，<code class="language-plaintext highlighter-rouge">valid.feature.txt</code>，<code class="language-plaintext highlighter-rouge">test.feature.txt</code>というファイル名で保存せよ．
なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．</p>


<h2 id="52-学習">52. 学習</h2>
<p>51で構築した学習データを用いて，ロジスティック回帰モデルを学習せよ．</p>


<h2 id="53-予測">53. 予測</h2>
<p>52で学習したロジスティック回帰モデルを用い，与えられた記事見出しからカテゴリとその予測確率を計算するプログラムを実装せよ．</p>


<h2 id="54-正解率の計測">54. 正解率の計測</h2>
<p>52で学習したロジスティック回帰モデルの正解率を，学習データおよび評価データ上で計測せよ．</p>


<h2 id="55-混同行列の作成">55. 混同行列の作成</h2>
<p>52で学習したロジスティック回帰モデルの混同行列（confusion matrix）を，学習データおよび評価データ上で作成せよ．</p>


<h2 id="56-適合率再現率f1スコアの計測">56. 適合率，再現率，F1スコアの計測</h2>
<p>52で学習したロジスティック回帰モデルの適合率，再現率，F1スコアを，評価データ上で計測せよ．カテゴリごとに適合率，再現率，F1スコアを求め，カテゴリごとの性能をマイクロ平均（micro-average）とマクロ平均（macro-average）で統合せよ．</p>


<h2 id="57-特徴量の重みの確認">57. 特徴量の重みの確認</h2>
<p>52で学習したロジスティック回帰モデルの中で，重みの高い特徴量トップ10と，重みの低い特徴量トップ10を確認せよ．</p>


<h2 id="58-正則化パラメータの変更">58. 正則化パラメータの変更</h2>
<p>ロジスティック回帰モデルを学習するとき，正則化パラメータを調整することで，学習時の過学習（overfitting）の度合いを制御できる．異なる正則化パラメータでロジスティック回帰モデルを学習し，学習データ，検証データ，および評価データ上の正解率を求めよ．実験の結果は，正則化パラメータを横軸，正解率を縦軸としたグラフにまとめよ．</p>


<h2 id="59-ハイパーパラメータの探索">59. ハイパーパラメータの探索</h2>
<p>学習アルゴリズムや学習パラメータを変えながら，カテゴリ分類モデルを学習せよ．検証データ上の正解率が最も高くなる学習アルゴリズム・パラメータを求めよ．また，その学習アルゴリズム・パラメータを用いたときの評価データ上の正解率を求めよ．</p>
